In [4]:
import itertools as it
import pandas as pd
import requests as rq
from IPython.display import display

In [5]:
# Parameters
experiments=1
users = "../data/users.json"
host = "localhost"
port = "5000"

sample_size = 500

hash_functions=["modulo"]
partitions=["0", "1"]
limits = [10]
walks = [1000]
walk_length = [100]

In [6]:
# Import users
user_df = pd.read_json(users, orient="table")
display(user_df)

# Compute cross product of all configuration values
configurations = list(it.product(hash_functions, partitions, walks, walk_length, limits))
configurations

,user
2267238,4671229608
1396952,2312687048
5066612,228541032
3807989,946097265625538560
2680420,1018562002678108162
...,...
748630,1597388371
1156305,2696695362
5014030,1330692817
962078,2987955914


[('modulo', '0', 1000, 100, 10), ('modulo', '1', 1000, 100, 10)]

In [14]:
def get_recommendations(user_df, walks, walk_length, limit, hash_function, partition):
    rows = []
#     port = port + int(partition) + 1
    partition_port = int(port) + int(partition) + 1

    for i, row in user_df.iterrows():
        user = row["user"]
        url = f"http://{host}:{partition_port}/recommendation/salsa/{user}?walks={walks}&walk_length={walk_length}&limit={limit}"
        response = rq.get(url)

        status_code = response.status_code
        duration = response.elapsed.total_seconds()
        recommendations = response.json() if response.status_code == 200 else []
        rows.append((walks, walk_length, limit, url, hash_function, partition, status_code, duration, user, recommendations))
    
    return pd.DataFrame(rows, columns=["walks", "walk_length", "limit", "url", "hash_function", "partition","status_code", "duration", "user", "recommendations"])

In [16]:
# Perform API requests

for experiment in range(experiments):

    offset = 0
    result_dfs = []

    for (hash_func, partition, walks, walk_length, limit) in configurations:
        print(
            f"Fetch recommendations: hash {hash_func},"  
            f"partition {partition}, walks {walks},"
            f" walk_length {walk_length}, limit {limit}"
        )
        result_dfs.append(get_recommendations(user_df, walks, walk_length, limit, hash_func, partition))
        offset = offset + sample_size

    result_df = pd.concat(result_dfs)
    display(result_df)

    # Save results
    output = f"../data/recommendations_experiment.json"
    result_df.to_json(output, index=False, orient="table")
    result_df.head(50)

Fetch recommendations: hash modulo,partition 0, walks 1000, walk_length 100, limit 10
Fetch recommendations: hash modulo,partition 1, walks 1000, walk_length 100, limit 10


,walks,walk_length,limit,url,hash_function,partition,status_code,duration,user,recommendations
0,1000,100,10,http://localhost:5001/recommendation/salsa/467...,modulo,0,200,0.006412,4671229608,[]
1,1000,100,10,http://localhost:5001/recommendation/salsa/231...,modulo,0,200,0.003921,2312687048,[]
2,1000,100,10,http://localhost:5001/recommendation/salsa/228...,modulo,0,200,0.462631,228541032,"[1133920658994409472, 1134302923226267648, 113..."
3,1000,100,10,http://localhost:5001/recommendation/salsa/946...,modulo,0,200,0.002265,946097265625538560,[]
4,1000,100,10,http://localhost:5001/recommendation/salsa/101...,modulo,0,200,0.668351,1018562002678108162,"[1134286895859490816, 1124998188513890306, 111..."
...,...,...,...,...,...,...,...,...,...,...
495,1000,100,10,http://localhost:5002/recommendation/salsa/159...,modulo,1,200,0.211339,1597388371,"[1133832886719635457, 1133081144633700353, 111..."
496,1000,100,10,http://localhost:5002/recommendation/salsa/269...,modulo,1,200,0.002579,2696695362,[]
497,1000,100,10,http://localhost:5002/recommendation/salsa/133...,modulo,1,200,0.001909,1330692817,[]
498,1000,100,10,http://localhost:5002/recommendation/salsa/298...,modulo,1,200,0.001842,2987955914,[]
